In [ ]:
# Dependencies
# %matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import time
from datetime import datetime, timedelta, date
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
pd.set_option('display.max_columns', None)

In [ ]:
#this is a timer function to see how long it takes to run the code
import time

def tic():
    """
    Start the timer.
    """
    global _start_time
    _start_time = time.time()

def toc():
    """
    Stop the timer and return the elapsed time in seconds.
    """
    if '_start_time' in globals():
        elapsed_time = time.time() - _start_time
        print("Elapsed time: {:.4f} seconds".format(elapsed_time))
        del globals()['_start_time']
    else:
        print("Timer hasn't been started.")

In [ ]:
acbn = pd.read_excel('all_combined.xlsx')

In [ ]:
# group the DataFrame by four columns and get the record with the most recent date
grouped = acbn.groupby(['Country']).agg({'Time': 'max'})
grouped
# merge the grouped DataFrame with the original DataFrame to get the full rows with the most recent dates
#result = pd.merge(grouped, acbn, on=['Level', 'Region', 'Country', 'State/Province', 'Date'], how='left')


In [ ]:
grouped.to_csv('latest_time_2.csv')

In [ ]:
acbn_other = acbn[acbn['Level'] != 'State/Province']
acbn_other.head()

In [ ]:
tic()
acbn_other.to_csv('all_combined_rest.csv')
toc()

In [ ]:
cnd_new_cb = pd.read_csv('cnd_new_div7_cb.csv')
cnd_new_cb.head()

In [ ]:
blank = pd.concat([acbn_other, cnd_new_cb], ignore_index=True)

In [ ]:
missing_cols1 = acbn_other.columns.difference(cnd_new_cb.columns)
missing_cols1

In [ ]:
for col in missing_cols1:
    blank[col] = blank[col].fillna(0)

In [ ]:
len(missing_cols1)

In [ ]:
usa_new = pd.read_csv('usa_new_fx_0516.csv')

In [ ]:
blank = pd.concat([blank, usa_new], ignore_index=True)

In [ ]:
missing_cols = blank.columns.difference(usa_new.columns)
print(len(missing_cols))

In [ ]:
for col in missing_cols:
    blank[col] = blank[col].fillna(0)

In [ ]:
blank[blank["State/Province"]=="Alberta"].tail()

In [ ]:
# Convert the "Time" column to datetime if it's not already in that format
blank['Time'] = pd.to_datetime(blank['Time'])

# Filter the dataframe to keep only the dates before May 1st, 2023
df_filtered = blank[blank['Time'] < pd.to_datetime('2023-05-01')]

In [ ]:
acbn_other['Time'] = pd.to_datetime(acbn_other['Time'])

# Filter the dataframe to keep only the dates before May 1st, 2023
acbno_filtered = acbn_other[acbn_other['Time'] < pd.to_datetime('2023-05-01')]

In [ ]:
acbno_filtered.tail()

In [ ]:
tic()
acbno_filtered.to_csv('rotw_beforeAPR30.csv', index=False)
toc()
#our data used about 14 seconds

In [ ]:
tic()
df_filtered.to_excel('full_beforeAPR30.xlsx', index=False)
toc()
#this xlsx file used about 680 seconds, which is about 11 minutes
#open an episode of Friends and you should be good to go

In [ ]:
tic()
df_filtered.to_csv('full_beforeAPR30.csv', index=False)
toc()
#this csv file used about 23 seconds